# Identify all markdown files in the master folder which contain folders of files

Functions that are useful to iterate through each file

In [14]:
import os

In [24]:
# Function that yields one file path at a time, ignoring .DS_Store and non-image files
def get_files_one_by_one(directory: str) -> str:
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Skip .DS_Store file
            if file == '.DS_Store':
                continue
            # Include only .md files
            if file.lower().endswith(".md"):
                yield os.path.join(root, file)

# Function that gets all file paths in a list by calling get_files_one_by_one
def get_file_paths(directory: str) -> list:
    return list(get_files_one_by_one(directory))

def get_file_name_and_parent_folder(file_path: str) -> (str, str):
    file_name = os.path.basename(file_path)  # Extracts the file name
    parent_folder = os.path.basename(os.path.dirname(file_path))  # Extracts the parent folder
    return parent_folder, file_name

In [18]:
list_of_md_files = get_file_paths("../data/ocr_results_outputs/")
list_of_md_files[:5]

['../data/ocr_results_outputs/document_1020/document_1020.md',
 '../data/ocr_results_outputs/document_904/document_904.md',
 '../data/ocr_results_outputs/document_138/document_138.md',
 '../data/ocr_results_outputs/document_597/document_597.md',
 '../data/ocr_results_outputs/document_107/document_107.md']

# Detect large tables in each file

In [ ]:
min_num_rows = 15

In [73]:
import re

In [77]:
def is_alignment_line(line):
    """Check if a line is a Markdown table alignment line."""
    return bool(re.match(r'^\s*\|?(\s*:?-+:?\s*\|)+\s*$', line))

def extract_markdown_tables(md_path):
    """Extract tables from a Markdown file using alignment lines and track start/end."""
    tables = []
    inside_table = False
    current_table = []
    start_index = None

    with open(md_path, 'r', encoding='utf-8') as md_file:
        lines = md_file.readlines()

    for i, line in enumerate(lines):
        stripped = line.strip()

        if is_alignment_line(stripped):
            # Alignment line means start of new table
            current_table = []

            # Optional header just above the alignment line
            if i > 0 and lines[i - 1].strip().startswith('|'):
                current_table.append((i - 1, lines[i - 1]))
                start_index = i - 1
            else:
                start_index = i

            current_table.append((i, line))
            inside_table = True

        elif inside_table:
            if stripped.startswith('|'):
                current_table.append((i, line))
            elif is_alignment_line(stripped):
                # Don't end table yet — alignment line means a new one is about to start
                # Save current table before continuing
                if current_table:
                    tables.append({
                        "start": start_index,
                        "end": current_table[-1][0],
                        "lines": current_table
                    })
                # Prepare to reprocess this line
                current_table = []
                inside_table = False
                start_index = None

                # Re-evaluate this line as the start of a new table in next loop
                continue
            else:
                # End of table
                if current_table:
                    tables.append({
                        "start": start_index,
                        "end": current_table[-1][0],
                        "lines": current_table
                    })
                inside_table = False
                current_table = []
                start_index = None

    # Final table at EOF
    if inside_table and current_table:
        tables.append({
            "start": start_index,
            "end": current_table[-1][0],
            "lines": current_table
        })

    return tables

In [79]:
md_path = "../data/ocr_results_outputs/document_932/document_932.md"
tables = extract_markdown_tables(md_path)

for idx, table in enumerate(tables, 1):
    print(f"\nTable {idx} (lines {table['start']} to {table['end']}):")
    for _, line in table["lines"]:
        print(line.rstrip())


Table 1 (lines 106 to 109):
| City | State | ZIP code |
| :-- | :--: | :--: |
| Primary | Primary |  |
| Phone | Email |  |

Table 2 (lines 171 to 177):
| Meter \# | 1010111635 |
| :-- | --: |
| Total Usage | 550.086100 kWh |
| Baseline Territory | X |
| Heat Source | B - Not Electric |
| Serial | B |
| Rotating Outage Block | 50 |

Table 3 (lines 196 to 199):
| Peak $^{1}$ | Usage | Energy Charges |
| :--: | :--: | :--: |
| Off Peak ${ }^{2}$ | 22.50\% | $\$ 48.52$ |
|  | 77.50\% | \$159.68 |

Table 4 (lines 204 to 218):
| Details of Silicon Valley Clean Energy Electric Generation Charges |  |  |
| :--: | :--: | :--: |
| 11/22/2022 - 12/21/2022 (30 billing days) |  |  |
| Service For: 400 E REMINGTON DR |  |  |
| Service Agreement ID: 4138890712 ESP Customer Number: 4134614813 |  |  |
| 11/22/2022 - 12/21/2022 |  |  |
| Rate Schedule: E-TOU-C |  |  |
| Generation - Off Peak - Winter | 426.280900 kWh @ \$0.11557 | \$49.27 |
| Generation - On Peak - Winter | 123.805200 kWh @ \$0.13044 

# Extract the table data

In [136]:
def table_to_rows(table_text: str) -> list:
    table_lines = table_text.splitlines()
    # Disregard the table if there are less than 2 rows (which could be header and alignment line.
    if len(table_lines) <= 2:
        return list()
    
    rows = []
    
    for table_line in table_lines:
        # Disregard the line which break between header and body
        if is_alignment_line(table_line):
            continue
            
        cleaned_row = table_line.strip('|').split('|')
        
        # Disregard row if all cells are empty
        if all([not cell.strip() for cell in cleaned_row]):
            continue
        
        rows.append(cleaned_row)    
        
    # Cleanup spcaing for each cell
    rows = [[cell.strip() for cell in row] for row in rows]
    
    return rows

In [127]:
table_to_rows("""| Your Electric Charges Breakdown (from page 2) |  |
| :--: | :--: |
| Conservation Incentive | $\$ 20.08$ |
| Transmission | 36.94 |
| Distribution | 75.47 |
| Electric Public Purpose Programs | 16.35 |
| Nuclear Decommissioning | $-0.10$ |
| Wildfire Fund Charge | 3.41 |
| Recovery Bond Charge | 8.27 |
| Recovery Bond Credit | $-8.27$ |
| Wildfire Hardening Charge | 1.05 |
| Competition Transition Charges (CTC) | 0.17 |
| Energy Cost Recovery Amount | $-3.19$ |
| PCIA | 15.01 |
| Taxes and Other | 4.01 |
| Total Electric Charges | $\$ 169.20$ |
""")

[['Your Electric Charges Breakdown (from page 2)', ''],
 ['Conservation Incentive', '$\\$ 20.08$'],
 ['Transmission', '36.94'],
 ['Distribution', '75.47'],
 ['Electric Public Purpose Programs', '16.35'],
 ['Nuclear Decommissioning', '$-0.10$'],
 ['Wildfire Fund Charge', '3.41'],
 ['Recovery Bond Charge', '8.27'],
 ['Recovery Bond Credit', '$-8.27$'],
 ['Wildfire Hardening Charge', '1.05'],
 ['Competition Transition Charges (CTC)', '0.17'],
 ['Energy Cost Recovery Amount', '$-3.19$'],
 ['PCIA', '15.01'],
 ['Taxes and Other', '4.01'],
 ['Total Electric Charges', '$\\$ 169.20$']]

# Save it as a CSV file 

# Reference the table in the Markdown